In [15]:
import glob

In [16]:
master=glob.glob("Data/*.htm")

In [17]:
master

['Data/Page 11 of 50 for Health Insurance Reviews by Ratings & City.htm',
 'Data/Page 2 of 50 for Health Insurance Reviews by Ratings & City.htm',
 'Data/Page 9 of 50 for Health Insurance Reviews by Ratings & City.htm',
 'Data/Page 5 of 50 for Health Insurance Reviews by Ratings & City.htm',
 'Data/Page 16 of 50 for Health Insurance Reviews by Ratings & City.htm',
 'Data/Page 20 of 50 for Health Insurance Reviews by Ratings & City.htm',
 'Data/Page 4 of 50 for Health Insurance Reviews by Ratings & City.htm',
 'Data/Page 17 of 50 for Health Insurance Reviews by Ratings & City.htm',
 'Data/Page 10 of 50 for Health Insurance Reviews by Ratings & City.htm',
 'Data/Page 3 of 50 for Health Insurance Reviews by Ratings & City.htm',
 'Data/Page 8 of 50 for Health Insurance Reviews by Ratings & City.htm',
 'Data/Health Insurance Reviews by Ratings & City.htm',
 'Data/Page 6 of 50 for Health Insurance Reviews by Ratings & City.htm',
 'Data/Page 15 of 50 for Health Insurance Reviews by Ratings & 

In [18]:
import codecs

In [19]:
S=" "
html_array=[]
for file in master:
    f=codecs.open(file, 'r')
    html_array.append(f.read())

In [20]:
html = S.join(html_array)

In [21]:
html.count("text_here review-desc-more")

400

In [22]:
from bs4 import BeautifulSoup

In [23]:
parsed_html=BeautifulSoup(html,"html.parser")

In [24]:
reviews=parsed_html.find_all('div',attrs={'class':"text_here review-desc-more",'itemprop':"description"})

In [25]:
reviews

[<div class="text_here review-desc-more" itemprop="description">
                                 I have taken a health insurance with ICICI LOMBARD GENERAL INSURANCE from my company on 4 years before. For the amount of 2 lakhs of sum assured amount. It covers my whole family.  It is yearly basis renewable. They have tied up with many hospitals except ESI. Cashless treatment also applicable. </div>,
 <div class="text_here review-desc-more" itemprop="description">
                                 The services are good from Apollo Munich and this is the reason for taking a health policy. Its been 4 years and need to renew the policy yearly once. The premium is around 6K through cheque. I do have claim experience. The policy coverage is 5L.</div>,
 <div class="text_here review-desc-more" itemprop="description">
                                 I BOUGHT FOR MY PARENTS AND POLICY NUMBER IS 13110445 WHEN I PURCHASED RELIGARE CARE PLAN LAST YEAR WITH PREMIUM 25282 BUT NOW MY PREMIUM PLUNGE LI

In [26]:
ratings=parsed_html.find_all('div',attrs={'class':'rating-section review-user-score',})

In [27]:
ratings

[<div class="rating-section review-user-score">
 <div class="medium-rating rating review-score-container" id="review.reviewRating.customer.0" title="Satisfactory">
 <span class="star-rating star-rating-3-0"></span>
 <input name="review.reviewRating.customer.0" type="hidden" value="3.0"/>
 </div> <span> <span class="dontshow">0.5</span>
 <span>3.0</span>/<span>5</span></span> "Satisfactory"
                             </div>,
 <div class="rating-section review-user-score">
 <div class="medium-rating rating review-score-container" id="review.reviewRating.customer.1" title="Great!">
 <span class="star-rating star-rating-4-0"></span>
 <input name="review.reviewRating.customer.1" type="hidden" value="4.0"/>
 </div> <span> <span class="dontshow">0.5</span>
 <span>4.0</span>/<span>5</span></span> "Great!"
                             </div>,
 <div class="rating-section review-user-score">
 <div class="medium-rating rating review-score-container" id="review.reviewRating.customer.2" title="Una

In [28]:
l=str(ratings[0])
float(l[(l.find('value=')+7):(l.find('value=')+10)])

3.0

In [29]:
import re

In [30]:
def cleanhtml(y):
    clean=re.compile('<.*?>')
    cleantext=re.sub(clean,'',y)
    return (cleantext.strip())

In [31]:
rews=[]
for i in range(0,len(reviews)):
    rews.append(cleanhtml(reviews[i].text))

In [32]:
rews

['I have taken a health insurance with ICICI LOMBARD GENERAL INSURANCE from my company on 4 years before. For the amount of 2 lakhs of sum assured amount. It covers my whole family.  It is yearly basis renewable. They have tied up with many hospitals except ESI. Cashless treatment also applicable.',
 'The services are good from Apollo Munich and this is the reason for taking a health policy. Its been 4 years and need to renew the policy yearly once. The premium is around 6K through cheque. I do have claim experience. The policy coverage is 5L.',
 'I BOUGHT FOR MY PARENTS AND POLICY NUMBER IS 13110445 WHEN I PURCHASED RELIGARE CARE PLAN LAST YEAR WITH PREMIUM 25282 BUT NOW MY PREMIUM PLUNGE LIKE HORSE BY 9570 Rs AND NOW IT IS 34852 .\nI HAVE MAILED MANY TIMES BUT THEY ALSO DONT KNOW HOW COMPANY DOES WORK... CUSTOMER SERVICE ALSO NOT GOOD THEY ALSO DONT KNOW HOW TO SPEAK WITH EXISTING CUSTOMER.',
 'Last 2 years, i have been renewing the health insurance policy from ICICI Lombard. I was a

In [33]:
def cleanrats(y):
    return(float(y[(y.find('value=')+7):(y.find('value=')+10)]))

In [34]:
rats=[]
for i in range(0,len(ratings)):
    rats.append(cleanrats(str(ratings[i])))

In [35]:
rats

[3.0,
 4.0,
 0.5,
 5.0,
 3.0,
 4.0,
 3.0,
 5.0,
 5.0,
 4.0,
 5.0,
 5.0,
 3.0,
 3.5,
 4.0,
 5.0,
 4.0,
 3.0,
 5.0,
 3.0,
 5.0,
 5.0,
 5.0,
 5.0,
 4.0,
 5.0,
 5.0,
 5.0,
 2.0,
 5.0,
 5.0,
 5.0,
 4.0,
 5.0,
 3.0,
 3.0,
 5.0,
 1.0,
 4.0,
 4.0,
 4.0,
 5.0,
 4.0,
 4.0,
 0.5,
 5.0,
 4.5,
 5.0,
 5.0,
 5.0,
 4.0,
 4.0,
 5.0,
 3.0,
 5.0,
 5.0,
 3.0,
 2.0,
 4.0,
 5.0,
 3.0,
 4.0,
 5.0,
 5.0,
 5.0,
 5.0,
 4.0,
 4.0,
 4.0,
 5.0,
 5.0,
 0.5,
 4.0,
 4.0,
 5.0,
 5.0,
 4.0,
 5.0,
 5.0,
 4.0,
 5.0,
 4.0,
 4.0,
 4.0,
 5.0,
 5.0,
 4.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 4.0,
 5.0,
 5.0,
 4.0,
 4.0,
 4.0,
 4.5,
 4.0,
 4.0,
 4.0,
 5.0,
 3.5,
 5.0,
 3.0,
 4.0,
 5.0,
 5.0,
 4.0,
 5.0,
 5.0,
 5.0,
 5.0,
 4.0,
 4.5,
 3.0,
 4.0,
 5.0,
 5.0,
 5.0,
 3.0,
 4.0,
 4.0,
 3.0,
 5.0,
 5.0,
 4.0,
 3.5,
 5.0,
 5.0,
 5.0,
 5.0,
 3.5,
 4.0,
 0.5,
 3.0,
 4.5,
 5.0,
 4.0,
 4.0,
 4.5,
 4.0,
 5.0,
 5.0,
 4.0,
 4.0,
 3.0,
 3.0,
 4.5,
 3.0,
 1.0,
 5.0,
 5.0,
 4.0,
 3.0,
 5.0,
 3.5,
 5.0,
 5.0,
 5.0,
 4.0,
 4.0,
 4.0,
 5.0

In [37]:
import pandas as pd

In [38]:
df=pd.DataFrame({'Reviews': rews, 'Ratings': rats})

In [39]:
df['Reviews']=df['Reviews'].replace(r'\n',' ',regex=True)

In [40]:
df['Reviews'].apply(lambda x:x.strip())

0      I have taken a health insurance with ICICI LOM...
1      The services are good from Apollo Munich and t...
2      I BOUGHT FOR MY PARENTS AND POLICY NUMBER IS 1...
3      Last 2 years, i have been renewing the health ...
4      I have taken a health insurance in IFFCO TOKIO...
5      Currently my health policy is in Star Health b...
6      I am having a health insurance from National. ...
7      From my working place, they gave me a health i...
8      I got a health insurance in United India Insur...
9      Since the premium amount is less and the benef...
10     I hold a health insurance from Star Health and...
11     For each person, they have provided a Rs. 2 la...
12     Actually star insurance has included many hosp...
13     I have a health insurance policy with Cigna TT...
14     When i purchased my credit card, they have off...
15     I have taken a life insurance in LIBERTY GENER...
16     My health insurance was selected from Health I...
17     From my employer side, t

In [41]:
df.dtypes

Reviews     object
Ratings    float64
dtype: object

In [42]:
def sentiment_calculation(y):
    if y['Ratings']<4.0:
        return 0
    else:
        return 1

In [43]:
df['Sentiment']=df.apply(sentiment_calculation,axis=1)

In [44]:
df.head(6)

,Reviews,Ratings,Sentiment
0,I have taken a health insurance with ICICI LOM...,3.0,0
1,The services are good from Apollo Munich and t...,4.0,1
2,I BOUGHT FOR MY PARENTS AND POLICY NUMBER IS 1...,0.5,0
3,"Last 2 years, i have been renewing the health ...",5.0,1
4,I have taken a health insurance in IFFCO TOKIO...,3.0,0
5,Currently my health policy is in Star Health b...,4.0,1


In [45]:
df.to_csv("Health insurance reviews.csv")

In [47]:
from sklearn.model_selection import train_test_split

In [48]:
x,y=df.Reviews, df.Sentiment

In [49]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [164]:
np.size(y_test)

132

In [108]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [149]:
# This is the function to contevert both the train data and the test data into feature vectors.
def tfidf_extractor(corpus, ngram_range=(1,1)):
    vectorizer=TfidfVectorizer(min_df=1,
                              norm='l2',
                              smooth_idf=True,
                              use_idf=True,
                              ngram_range=ngram_range)
    features=vectorizer.fit_transform(corpus)
    return vectorizer, features

In [150]:
tfidf_vectorizer, tfidf_train_features=tfidf_extractor(x_train)
tfidf_test_features=tfidf_vectorizer.transform(x_test)

In [151]:
(tfidf_test_features)

<132x1074 sparse matrix of type '<class 'numpy.float64'>'
	with 5400 stored elements in Compressed Sparse Row format>

In [152]:
tfidf_train_features.data

array([0.11639807, 0.07429508, 0.10762987, ..., 0.1188461 , 0.05510784,
       0.06465765])

In [153]:
tfidf_test_features.data

array([0.28936975, 0.71068585, 0.16115809, ..., 0.23685743, 0.30198511,
       0.1454835 ])

In [171]:
import numpy as np

In [172]:
from sklearn import metrics

In [183]:
def get_metrics(true_labels, predicted_labels):
    print("Accuracy:",np.round(metrics.accuracy_score(true_labels, predicted_labels),2))
    print("Precision:",np.round(metrics.precision_score(true_labels, predicted_labels,average='weighted'),2))
    print("Recall:",np.round(metrics.recall_score(true_labels, predicted_labels,average='weighted'),2))
    print("f1:",np.round(metrics.f1_score(true_labels, predicted_labels,average='weighted'),2))

In [184]:
def train_test_eval_model(classifier, train_features, train_labels, test_features, test_labels):
    classifier.fit(train_features,train_labels)
    predictions=classifier.predict(test_features)
    print(type(predictions))
    get_metrics(true_labels=test_labels, predicted_labels=predictions)
    return predictions

In [185]:
from sklearn.naive_bayes import MultinomialNB

In [186]:
mnb_best=MultinomialNB(alpha=0.001, fit_prior=True)

In [187]:
mnb_tfidf_predictions= train_test_eval_model(classifier=mnb_best,
                                            train_features=tfidf_train_features,
                                            train_labels=y_train,
                                            test_features=tfidf_test_features,
                                            test_labels=y_test)

<class 'numpy.ndarray'>
Accuracy: 0.8
Precision: 0.74
Recall: 0.8
f1: 0.73


In [165]:
np.size(mnb_tfidf_predictions)

132

In [163]:
print(mnb_tfidf_predictions)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1
 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1]


In [167]:
from sklearn.metrics import classification_report

In [169]:
print(classification_report(y_test,mnb_tfidf_predictions))

              precision    recall  f1-score   support

           0       0.50      0.07      0.13        27
           1       0.80      0.98      0.88       105

    accuracy                           0.80       132
   macro avg       0.65      0.53      0.51       132
weighted avg       0.74      0.80      0.73       132



In [188]:
from sklearn.metrics import confusion_matrix

In [189]:
print(confusion_matrix(y_test,mnb_tfidf_predictions))

[[  2  25]
 [  2 103]]
